In [1]:
import Data_Conversion
import Input_Parameters 
import Data_Conversion
import Passive_Model
import Solar_Generation
import Electrical_Load
import RO

/Users/jimmy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import numpy as np
import pandas as pd

In [3]:
# Sensitivity Analysis on Loss of Load Cost, Stochastic Optimization
# Fred Fan, Stanford University, 03/17/2025

In [4]:
# Define the base data directory, list of loss of load costs, and the weather year.
data_dir = "Data"

location = "HalfMoonBay"

# Example loss of load cost sensitivity
lol_costs = [
             10, 
             100, 
             1000
             ] # $/kWh 

# 20 years of training data
weather_year_list_training = list(range(1998, 2018))  # use list(range(1998, 2018)) for full dataset
# 5 years of training data
weather_year_list_testing = list(range(2018, 2023))  # use list(range(2018, 2023)) for full dataset

# Set a random seed for consistency

total_cells = len(weather_year_list_training) + len(weather_year_list_testing) 

sequential_numbers = np.arange(1, total_cells + 1)

# Reshape the array into a 2D array with i rows and j columns
data = sequential_numbers.reshape(1, total_cells)

# Convert the array into a DataFrame
random_seeds = pd.DataFrame(data, columns=[f'Column_{k+1}' for k in range(total_cells)])

lat, lon, timezone = Data_Conversion.get_timezone_singlezone(data_dir, location)

print(random_seeds)

Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_1998.csv
   Column_1  Column_2  Column_3  Column_4  Column_5  Column_6  Column_7  \
0         1         2         3         4         5         6         7   

   Column_8  Column_9  Column_10  ...  Column_16  Column_17  Column_18  \
0         8         9         10  ...         16         17         18   

   Column_19  Column_20  Column_21  Column_22  Column_23  Column_24  Column_25  
0         19         20         21         22         23         24         25  

[1 rows x 25 columns]


In [5]:
# Create the dictionary
training_results = {}

In [6]:
# Sensitivity Analysis on Loss of Load Cost

# Initialize storing space to store all input_dfs
input_df_list_train = []

# First, collect all training data
for j in range(len(weather_year_list_training)):
    year = weather_year_list_training[j]

    # Get a unique random seed number
    random_seed = random_seeds.iloc[0, j]
    print(random_seed)
    
    # Read NSRDB weather data of the given location of the given year
    NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

    # Prepare weather data file using NSRDB data
    weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lat, lon, timezone)

    # Prepare heating and cooling load using weather data and passive model
    NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lat)

    # Prepare solar PV capacity factor using weather data
    pv_cf = Solar_Generation.generate_pv(weather_data, lat)

    # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
    load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
    
    # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
    input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)

    # Add input_df to list
    input_df_list_train.append(input_df)

# Now that we have all training data, run sensitivity analysis
training_results = {}  # Initialize results dictionary
for i in range(len(lol_costs)):
    lolc = lol_costs[i]
    print(f"Running optimization with loss of load cost: {lolc}")
    
    PV_Size, Battery_Size, PCM_Heating_Size, PCM_Cooling_Size, ObjValue, First_stage_cost, Second_stage_cost = RO.RO_training(input_df_list_train, lolc)

    # Store the variables in the nested dictionary
    training_results[lolc] = {
        'PV_Size': PV_Size,
        'Battery_Size': Battery_Size,
        'PCM_Heating_Size': PCM_Heating_Size,
        'PCM_Cooling_Size': PCM_Cooling_Size,
        'Total Cost': ObjValue,
        'Capital Cost': First_stage_cost,
        'Operation Cost': Second_stage_cost
    }

1
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_1998.csv
2
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_1999.csv
3
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2000.csv
4
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2001.csv
5
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2002.csv
6
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2003.csv
7
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2004.csv
8
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2005.csv
9
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2006.csv
10
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2007.csv
11
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2008.csv
12
Readi

In [7]:
print(training_results)

{10: {'PV_Size': 9.757, 'Battery_Size': 1.407, 'PCM_Heating_Size': 43.034, 'PCM_Cooling_Size': 0.0, 'Total Cost': 20468.287, 'Capital Cost': 19948.714, 'Operation Cost': 519.573}, 100: {'PV_Size': 11.352, 'Battery_Size': 3.927, 'PCM_Heating_Size': 59.344, 'PCM_Cooling_Size': 0.0, 'Total Cost': 21330.894, 'Capital Cost': 20777.48, 'Operation Cost': 553.414}, 1000: {'PV_Size': 14.404, 'Battery_Size': 6.8, 'PCM_Heating_Size': 65.871, 'PCM_Cooling_Size': 0.0, 'Total Cost': 21501.842, 'Capital Cost': 21501.842, 'Operation Cost': 0.0}}


In [8]:
# Testing:

# Create the nested dictionary
testing_results = {lolc: {year: {} for year in weather_year_list_testing} for lolc in lol_costs}

for j in range(len(weather_year_list_testing)):
    
    year = weather_year_list_testing[j]
    
    # Get a unique random seed number
    random_seed = random_seeds.iloc[0, j+len(weather_year_list_training)]
    print(random_seed)
    
    # Read NSRDB weather data of the given location of the given year
    NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

    # Prepare weather data file using NSRDB data
    weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lat, lon, timezone)

    # Prepare heating and cooling load using weather data and passive model
    NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lat)

    # Prepare solar PV capacity factor using weather data
    pv_cf = Solar_Generation.generate_pv(weather_data, lat)

    # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
    load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
    
    # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
    input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
    
    for i in range(len(lol_costs)):
        
        lolc = lol_costs[i]
        
        test_capacities = [training_results[lolc]['PV_Size'], training_results[lolc]['Battery_Size'], training_results[lolc]['PCM_Heating_Size'], training_results[lolc]['PCM_Cooling_Size']]
        
        ObjValue, First_stage_cost, Second_stage_cost = RO.simulate(input_df, lolc, test_capacities)
        # Store the variables in the nested dictionary
        testing_results[lolc][year] = {
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }

21
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_2018.csv
Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2025-04-15
Read LP format model from file /var/folders/ll/cq_mlqg56938kmwh8dys19n80000gn/T/tmpa8de0_bu.pyomo.lp
Reading time = 0.31 seconds
x1: 157683 rows, 140160 columns, 385434 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads
Optimize a model with 157683 rows, 140160 columns and 385434 nonzeros
Model fingerprint: 0xe2a96078
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-04, 4e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 96207 rows and 55661 columns
Presol

In [9]:
# Initialize a dictionary to store averaged results
testing_averaged_results = {}

# Iterate over each loss of load cost
for lolc, years_data in testing_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the loss of load cost
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages
    testing_averaged_results[lolc] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(testing_averaged_results)

{10: {'Total Cost': 20114.9298, 'Capital Cost': 19948.714, 'Operation Cost': 166.2152}, 100: {'Total Cost': 20908.5922, 'Capital Cost': 20777.48, 'Operation Cost': 131.11200000000002}, 1000: {'Total Cost': 21501.842, 'Capital Cost': 21501.842, 'Operation Cost': 0.0}}


In [10]:
from pathlib import Path

# Define the output file path
output_file = "Results_SA_LoLC_RO.xlsx"

# Store Training Result
train_df = pd.DataFrame(training_results).T 
sheet_name_train = "Training Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"File '{output_file}' created with sheet '{sheet_name_train}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_train}'.")


# Store Testing Result
test_df = pd.DataFrame(testing_averaged_results).T 
sheet_name_test = "Testing Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"File '{output_file}' created with sheet '{sheet_name_test}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_test}'.")

File 'Results_SA_LoLC_RO.xlsx' created with sheet 'Training Results'.
Data written to 'Results_SA_LoLC_RO.xlsx' in sheet 'Testing Results'.
